In [38]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import unidecode
import time
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [29]:
def obtain_links_periodos(driver):
    links_asistencias = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('votaciones', link)):
            links_asistencias.append(link)
    return(links_asistencias)

def obtain_links_votaciones(driver):
    links_asistencias_sesion = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('estadistico_votacion', link)):
            links_asistencias_sesion.append(link)
    return(links_asistencias_sesion)

def obtain_partyVotaciones(driver):
    links_asistencias_sesion = []
    partidos = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('a'):
        link = i.get_attribute('href')
        if any(re.findall('listados_votaciones', link)):
            links_asistencias_sesion.append(link)
            partidos.append(i.text)
    return(links_asistencias_sesion, partidos)

In [46]:
def month_to_num(x):
    if x == 'Enero':
        return(1)
    elif x == 'Febrero':
        return(2)
    elif x == 'Marzo':
        return(3)
    elif x == 'Abril':
        return(4)
    elif x == 'Mayo':
        return(5)
    elif x == 'Junio':
        return(6)
    elif x == 'Julio':
        return(7)
    elif x == 'Agosto':
        return(8)
    elif x == 'Septiembre':
        return(9)
    elif x == 'Octubre':
        return(10)
    elif x == 'Noviembre':
        return(11)
    elif x == 'Diciembre':
        return(12)

def obtain_votacion(driver, legislatura, partido):
    temp = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > b').text.split('\n')[1].split(' ')[-1:][0]
    date_session = datetime.date(int(temp.split('-')[2]), month_to_num(temp.split('-')[1]), int(temp.split('-')[0]))
    titulo_votacion = driver.find_element_by_css_selector('body > table > tbody > tr:nth-child(4) > td > table > tbody > tr:nth-child(2) > td > b').text.split('\n')[2]
    
    
    year_legis = []
    month_legis = []
    day_legis = []
    members = []
    party = []
    sentido_votacion = []
    legis = []
    titulo = []
    for i in driver.find_element_by_css_selector('table').find_elements_by_css_selector('tr')[10:]:
        try:
            if any(re.findall('[a-z]+', i.find_elements_by_css_selector('td')[0].text)):
                continue
            else:
                members.append(unidecode.unidecode(i.find_elements_by_css_selector('td')[1].text))
                sentido_votacion.append(i.find_elements_by_css_selector('td')[2].text)
                titulo.append(titulo_votacion)
                party.append(partido)
                year_legis.append(date_session.year)
                month_legis.append(date_session.month)
                day_legis.append(date_session.day)
                legis.append(legislatura)
        except:
            continue
            
    return(pd.DataFrame({'legislatura':legis, 
                 'year_session':year_legis, 
                 'month_session':month_legis, 
                 'day_session':day_legis, 
                 'member':members, 
                 'party':party, 
                 'voto':sentido_votacion,
                         'titulo_votacion':titulo
                 }))



In [45]:
links_legislaturas = [
    'http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_periodonplxiv.php',
    'http://sitllxiii.diputados.gob.mx/votaciones_por_periodonplxiii.php',
    'http://sitllxii.diputados.gob.mx/votaciones_por_periodonplxii.php', 
    'http://sitllxi.diputados.gob.mx/votaciones_por_periodonplxi.php', 
    'http://sitllx.diputados.gob.mx/votaciones_por_periodonp.php'
]

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
for link, leg in tqdm(zip(links_legislaturas, [64, 63, 62, 61, 60]), total = 5):
    data_legislatura = []
    driver.get(link)
    links_periodos = obtain_links_periodos(driver)
    for link2 in tqdm(links_periodos):
        driver.get(link2)
        links_sesiones = obtain_links_votaciones(driver)
        for link3 in tqdm(links_sesiones):
            driver.get(link3)
            links_sesiones_partido, partidos = obtain_partyVotaciones(driver)
            for link4, p in tqdm(zip(links_sesiones_partido, partidos), total = len(links_sesiones_partido)):
                driver.get(link4)
                data_legislatura.append(obtain_votacion(driver, leg, p))
    df_attendance_leg = pd.concat(data_legislatura)
    name_file = 'roll_call' + str(leg) + '.csv'
    df_attendance_leg.to_csv('../../data/02-roll_call_votes/' + name_file, encoding = 'utf-8')

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:40<05:24, 40.54s/it]


 22%|██▏       | 2/9 [00:53<03:46, 32.35s/it]


 33%|███▎      | 3/9 [01:03<02:34, 25.69s/it]


 44%|████▍     | 4/9 [01:14<01:45, 21.09s/it]


 56%|█████▌    | 5/9 [01:20<01:05, 16.49s/it]


 67%|██████▋   | 6/9 [01:24<00:38, 12.84s/it]


 78%|███████▊  | 7/9 [01:26<00:19,  9.75s/it]


 89%|████████▉ | 8/9 [01:29<00:07,  7.48s/it]


100%|██████████| 9/9 [01:29<00:00,  9.99s/it]


  3%|▎         | 1/37 [01:30<54:20, 90.56s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:48<06:25, 48.21s/it]


 22%|██▏       | 2/9 [01:03<04:27, 38.23s/it]


 33%|███▎      | 3/9 [01:10<02:54, 29.10s/it]


 44%|████▍     | 4/9 [01:19<01:55, 23.04s/it]


 56%|█████▌    | 5/9 [01:25<01:11, 17.76s/it]


 67%|██████▋   | 6/9 [01:29<00:41, 13.72s/it]


 78%|███████▊  | 7/9 [01:32<00:20, 10.43s/it]


 89%|████████▉ | 8/9 [0

  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:39<05:17, 39.70s/it]


 22%|██▏       | 2/9 [00:56<03:50, 32.88s/it]


 33%|███▎      | 3/9 [01:06<02:35, 25.87s/it]


 44%|████▍     | 4/9 [01:12<01:40, 20.08s/it]


 56%|█████▌    | 5/9 [01:16<01:00, 15.20s/it]


 67%|██████▋   | 6/9 [01:19<00:34, 11.65s/it]


 78%|███████▊  | 7/9 [01:21<00:17,  8.71s/it]


 89%|████████▉ | 8/9 [01:23<00:06,  6.66s/it]


100%|██████████| 9/9 [01:24<00:00,  9.38s/it]


 38%|███▊      | 14/37 [21:20<34:20, 89.58s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:39<05:13, 39.15s/it]


 22%|██▏       | 2/9 [00:52<03:40, 31.53s/it]


 33%|███▎      | 3/9 [01:01<02:28, 24.77s/it]


 44%|████▍     | 4/9 [01:09<01:38, 19.62s/it]


 56%|█████▌    | 5/9 [01:13<00:59, 14.95s/it]


 67%|██████▋   | 6/9 [01:17<00:34, 11.60s/it]


 78%|███████▊  | 7/9 [01:19<00:17,  8.73s/it]


 89%|████████▉ | 8/9 [01:21<00:06,  6.68s/it]


100%|██████████| 9/9 [01:22<00:00,  9.15s/it]


 41%|█

 22%|██▏       | 2/9 [00:51<03:31, 30.19s/it]


 33%|███▎      | 3/9 [01:00<02:23, 23.90s/it]


 44%|████▍     | 4/9 [01:08<01:35, 19.12s/it]


 56%|█████▌    | 5/9 [01:13<00:59, 14.79s/it]


 67%|██████▋   | 6/9 [01:17<00:34, 11.52s/it]


 78%|███████▊  | 7/9 [01:19<00:17,  8.75s/it]


 89%|████████▉ | 8/9 [01:21<00:06,  6.67s/it]


100%|██████████| 9/9 [01:22<00:00,  9.13s/it]


 73%|███████▎  | 27/37 [40:27<14:46, 88.66s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:42<05:42, 42.85s/it]


 22%|██▏       | 2/9 [00:55<03:56, 33.72s/it]


 33%|███▎      | 3/9 [01:05<02:40, 26.73s/it]


 44%|████▍     | 4/9 [01:14<01:47, 21.40s/it]


 56%|█████▌    | 5/9 [01:20<01:06, 16.68s/it]


 67%|██████▋   | 6/9 [01:24<00:38, 12.84s/it]


 78%|███████▊  | 7/9 [01:26<00:19,  9.65s/it]


 89%|████████▉ | 8/9 [01:28<00:07,  7.40s/it]


100%|██████████| 9/9 [01:29<00:00,  9.97s/it]


 76%|███████▌  | 28/37 [41:57<13:23, 89.24s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%

  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [01:53<15:08, 113.61s/it]


 22%|██▏       | 2/9 [02:17<10:06, 86.62s/it] 


 33%|███▎      | 3/9 [02:33<06:32, 65.47s/it]


 44%|████▍     | 4/9 [02:57<04:25, 53.06s/it]


 56%|█████▌    | 5/9 [03:08<02:42, 40.50s/it]


 67%|██████▋   | 6/9 [03:21<01:36, 32.13s/it]


 78%|███████▊  | 7/9 [03:31<00:51, 25.64s/it]


 89%|████████▉ | 8/9 [03:40<00:20, 20.53s/it]


100%|██████████| 9/9 [03:44<00:00, 24.93s/it]


  1%|          | 1/114 [03:45<7:05:30, 225.93s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [02:09<17:15, 129.46s/it]


 22%|██▏       | 2/9 [03:08<12:37, 108.19s/it]


 33%|███▎      | 3/9 [03:33<08:19, 83.29s/it] 


 44%|████▍     | 4/9 [03:48<05:15, 63.03s/it]


 56%|█████▌    | 5/9 [04:05<03:16, 49.20s/it]


 67%|██████▋   | 6/9 [04:17<01:53, 37.85s/it]


 78%|███████▊  | 7/9 [04:21<00:55, 27.73s/it]


 89%|████████▉ | 8/9 [04:24<00:20, 20.43s/it]


100%|██████████| 9/9 [04:26<00:00, 29.61s/it]


...

 44%|████▍     | 4/9 [00:54<01:18, 15.65s/it]


 56%|█████▌    | 5/9 [00:58<00:48, 12.23s/it]


 67%|██████▋   | 6/9 [01:02<00:28,  9.61s/it]


 78%|███████▊  | 7/9 [01:04<00:14,  7.32s/it]


 89%|████████▉ | 8/9 [01:06<00:05,  5.67s/it]


100%|██████████| 9/9 [01:07<00:00,  7.47s/it]


 46%|████▋     | 25/54 [31:18<35:00, 72.41s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:35<04:41, 35.15s/it]


 22%|██▏       | 2/9 [00:45<03:12, 27.57s/it]


 33%|███▎      | 3/9 [00:51<02:08, 21.36s/it]


 44%|████▍     | 4/9 [00:58<01:24, 16.94s/it]


 56%|█████▌    | 5/9 [01:02<00:52, 13.10s/it]


 67%|██████▋   | 6/9 [01:06<00:30, 10.20s/it]


 78%|███████▊  | 7/9 [01:08<00:15,  7.71s/it]


 89%|████████▉ | 8/9 [01:09<00:05,  5.93s/it]


100%|██████████| 9/9 [01:10<00:00,  7.86s/it]


 48%|████▊     | 26/54 [32:29<33:39, 72.13s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:32<04:17, 32.25s/it]


 22%|██▏       | 2/9 [00:42<02:59, 25.58s/it]


 33%

 67%|██████▋   | 6/9 [01:05<00:29,  9.93s/it]


 78%|███████▊  | 7/9 [01:07<00:15,  7.53s/it]


 89%|████████▉ | 8/9 [01:09<00:05,  5.88s/it]


100%|██████████| 9/9 [01:10<00:00,  7.78s/it]


 70%|███████   | 38/54 [46:50<18:49, 70.60s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:30<04:00, 30.08s/it]


 22%|██▏       | 2/9 [00:40<02:48, 24.04s/it]


 33%|███▎      | 3/9 [00:46<01:52, 18.81s/it]


 44%|████▍     | 4/9 [00:52<01:14, 14.90s/it]


 56%|█████▌    | 5/9 [00:56<00:46, 11.60s/it]


 67%|██████▋   | 6/9 [00:59<00:27,  9.15s/it]


 78%|███████▊  | 7/9 [01:01<00:13,  6.94s/it]


 89%|████████▉ | 8/9 [01:03<00:05,  5.45s/it]


100%|██████████| 9/9 [01:04<00:00,  7.15s/it]


 72%|███████▏  | 39/54 [47:55<17:13, 68.92s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:33<04:29, 33.67s/it]


 22%|██▏       | 2/9 [00:45<03:09, 27.03s/it]


 33%|███▎      | 3/9 [00:52<02:06, 21.10s/it]


 44%|████▍     | 4/9 [00:58<01:23, 16.71s/it]


 56%

 89%|████████▉ | 8/9 [01:09<00:05,  5.84s/it]


100%|██████████| 9/9 [01:10<00:00,  7.78s/it]


 94%|█████████▍| 51/54 [1:02:29<03:43, 74.55s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:33<04:30, 33.77s/it]


 22%|██▏       | 2/9 [00:45<03:09, 27.08s/it]


 33%|███▎      | 3/9 [00:52<02:07, 21.23s/it]


 44%|████▍     | 4/9 [00:59<01:24, 16.98s/it]


 56%|█████▌    | 5/9 [01:03<00:52, 13.10s/it]


 67%|██████▋   | 6/9 [01:07<00:30, 10.15s/it]


 78%|███████▊  | 7/9 [01:09<00:15,  7.90s/it]


 89%|████████▉ | 8/9 [01:12<00:06,  6.22s/it]


100%|██████████| 9/9 [01:13<00:00,  8.15s/it]


 96%|█████████▋| 52/54 [1:03:43<02:28, 74.38s/it]


  0%|          | 0/9 [00:00<?, ?it/s]


 11%|█         | 1/9 [00:33<04:28, 33.62s/it]


 22%|██▏       | 2/9 [00:43<03:06, 26.58s/it]


 33%|███▎      | 3/9 [00:50<02:03, 20.54s/it]


 44%|████▍     | 4/9 [00:55<01:20, 16.09s/it]


 56%|█████▌    | 5/9 [00:59<00:49, 12.42s/it]


 67%|██████▋   | 6/9 [01:09<00:35, 11.73s/it]




  9%|▉         | 5/53 [06:17<1:00:21, 75.44s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:24<03:44, 24.90s/it]


 20%|██        | 2/10 [00:37<02:50, 21.33s/it]


 30%|███       | 3/10 [00:44<01:58, 16.89s/it]


 40%|████      | 4/10 [00:52<01:25, 14.20s/it]


 50%|█████     | 5/10 [00:57<00:57, 11.53s/it]


 60%|██████    | 6/10 [01:00<00:36,  9.01s/it]


 70%|███████   | 7/10 [01:03<00:20,  6.98s/it]


 80%|████████  | 8/10 [01:04<00:10,  5.45s/it]


 90%|█████████ | 9/10 [01:05<00:04,  4.02s/it]


100%|██████████| 10/10 [01:06<00:00,  6.69s/it]


 11%|█▏        | 6/53 [07:25<57:14, 73.08s/it]  


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:25<03:50, 25.59s/it]


 20%|██        | 2/10 [00:39<02:57, 22.23s/it]


 30%|███       | 3/10 [00:46<02:03, 17.59s/it]


 40%|████      | 4/10 [00:53<01:25, 14.28s/it]


 50%|█████     | 5/10 [00:58<00:58, 11.64s/it]


 60%|██████    | 6/10 [01:02<00:36,  9.16s/it]


 70%|███████   | 7/10 [01:04<00

100%|██████████| 10/10 [01:11<00:00,  7.18s/it]


 32%|███▏      | 17/53 [20:23<43:00, 71.67s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:30<04:38, 30.90s/it]


 20%|██        | 2/10 [00:46<03:29, 26.21s/it]


 30%|███       | 3/10 [00:52<02:21, 20.25s/it]


 40%|████      | 4/10 [00:59<01:37, 16.27s/it]


 50%|█████     | 5/10 [01:04<01:04, 12.97s/it]


 60%|██████    | 6/10 [01:07<00:39,  9.98s/it]


 70%|███████   | 7/10 [01:09<00:22,  7.57s/it]


 80%|████████  | 8/10 [01:11<00:11,  5.81s/it]


 90%|█████████ | 9/10 [01:12<00:04,  4.25s/it]


100%|██████████| 10/10 [01:13<00:00,  7.35s/it]


 34%|███▍      | 18/53 [21:37<42:15, 72.44s/it]


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [00:26<03:58, 26.47s/it]


 20%|██        | 2/10 [00:40<03:02, 22.84s/it]


 30%|███       | 3/10 [00:48<02:07, 18.27s/it]


 40%|████      | 4/10 [00:55<01:28, 14.82s/it]


 50%|█████     | 5/10 [01:00<00:59, 11.97s/it]


 60%|██████    | 6/10 [01:04<00: